## Info

This is a slightly modified version of 

`Build Data Multi-Asset ETFs.ipynb`

In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,50026094,572926853120,The trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,7395415,60435103744,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,18043080,16526614528,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,21723,235239184,The fund generally will invest at least 90% of...
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,49670,687488448,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,23767843,14188549120,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,465424,1206291584,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,3354686,3193291776,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,7319141,34838351872,The underlying index measures the performance ...


In [5]:
STARTDATE = '2009-12-31'
ENDDATE = '2025-04-30'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

rets = prices.pct_change().dropna()
# reorder columns to match info tab, not alphabetical
rets = rets[info.index]

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_29853/2262798614.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,SPY,EFA,EEM,PSP,QAI,HYG,DBC,IYR,IEF,BWX,TIP,SHV
Date,,,,,,,,,,,,
2010-01-31,-0.036342,-0.050651,-0.077590,-0.003329,-0.015850,-0.018329,-0.078392,-0.054225,0.023702,-0.014605,0.015303,0.000272
2010-02-28,0.031194,0.002667,0.017764,0.021158,-0.000374,0.018734,0.041428,0.054571,0.003318,0.002418,-0.013031,0.000018
2010-03-31,0.060879,0.063854,0.081109,0.079607,0.019108,0.021335,-0.004655,0.097485,-0.010568,-0.000179,-0.000058,-0.000109
2010-04-30,0.015470,-0.028046,-0.001662,0.018182,-0.001838,0.019245,0.039116,0.063881,0.016527,-0.014474,0.025150,-0.000063
2010-05-31,-0.079455,-0.111928,-0.093936,-0.128968,-0.041621,-0.046756,-0.101473,-0.056835,0.029331,-0.036446,0.001917,0.000145
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,-0.024060,-0.029502,-0.016967,-0.051399,-0.015700,-0.007803,0.017227,-0.082892,-0.022584,-0.033754,-0.017144,0.004031
2025-01-31,0.026856,0.048009,0.021521,0.067995,0.016237,0.013605,0.027596,0.018913,0.006166,0.000935,0.013702,0.003633
2025-02-28,-0.012695,0.029531,0.011470,-0.040107,-0.005952,0.009703,0.001366,0.037861,0.027997,0.013991,0.021572,0.003115


In [8]:
TICKS_PORT = ['PSP','QAI','IYR','IEF']
port = (rets[TICKS_PORT].mean(axis=1)).to_frame().rename(columns={0:'portfolio'})

In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'info')
    #prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    port.to_excel(writer, sheet_name='portfolio returns')
    #retsx.to_excel(writer, sheet_name='excess returns')